In [202]:
#libraries

import pandas as pd
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np


In [203]:
#load data

import pandas as pd

patients = pd.read_csv("patients.csv")
services = pd.read_csv("services_weekly.csv")
staff = pd.read_csv("staff.csv")
schedule = pd.read_csv("staff_schedule.csv") 



In [204]:
cols = [
    "available_beds",
    "staff_morale",
    "patient_satisfaction",
    "patients_request",
    "patients_admitted",
    "patients_refused"
]


In [205]:
    
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# services_list = services["service"].unique()

# fig = make_subplots(
#     rows=2, cols=2,
#     subplot_titles=services_list,
#     horizontal_spacing=0.14,
#     vertical_spacing=0.16
# )

# for i, service in enumerate(services_list):
#     subset = services[services["service"] == service]
#     corr = subset[cols].corr()

#     r = i // 2 + 1
#     c = i % 2 + 1

#     fig.add_trace(
#         go.Heatmap(
#             z=corr.values,
#             x=corr.columns,
#             y=corr.columns,
#             zmin=-1, zmax=1,
#             colorscale="RdBu_r",
#             showscale=(i == 0),
#             # text=corr.round(2).values,
#             # texttemplate="<b>%{text}</b>",
#             # textfont=dict(size=9)
            
#         ),
#         row=r, col=c
#     )

# # Fix overlapping labels
# fig.update_xaxes(tickangle=-45, automargin=True, tickfont=dict(size=8, family="Arial Black"))
# fig.update_yaxes(automargin=True, tickfont=dict(size=8, family="Arial Black"))

# # Improve layout
# fig.update_layout(
#     title=dict(
#         text="<b>Correlation Heatmaps for each service</b>",
#         x=0.5,
#         xanchor="center"
#     ),
#     height=850,
#     width=950
# )

# fig.show()


   

In [206]:
#CORRELATION HEATMAP

services_list = services["service"].unique()
weeks = sorted(services["week"].unique()) 


# Subplots as before
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=services_list,
    horizontal_spacing=0.14,
    vertical_spacing=0.16
)

# --- Initial data for the first week value ---
initial_week = weeks[0]

for i, service in enumerate(services_list):
    subset = services[
        (services["service"] == service) &
        (services["week"] <= initial_week)
    ]
    corr = subset[cols].corr()

    r = i // 2 + 1
    c = i % 2 + 1

    fig.add_trace(
        go.Heatmap(
            z=corr.values,
            x=corr.columns,
            y=corr.columns,
            zmin=-1, zmax=1,
            colorscale="RdBu_r",
            showscale=(i == 0),  # colorbar only on first subplot
        ),
        row=r, col=c
    )

# --- Build frames: one set of heatmaps per week value ---
frames = []
for w in weeks:
    frame_data = []
    for i, service in enumerate(services_list):
        subset = services[
            (services["service"] == service) &
            (services["week"] <= w)
        ]
        corr = subset[cols].corr()

        frame_data.append(
            go.Heatmap(
                z=corr.values,
                x=corr.columns,
                y=corr.columns,
                zmin=-1, zmax=1,
                colorscale="RdBu_r",
                showscale=(i == 0),
            )
        )

    frames.append(go.Frame(data=frame_data, name=str(w)))

fig.frames = frames

# --- Slider under the plots ---
slider_steps = []
for w in weeks:
    slider_steps.append(
        dict(
            method="animate",
            label=str(w),
            args=[
                [str(w)],  # name of the frame
                {
                    "frame": {"duration": 0, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 0},
                },
            ],
        )
    )

sliders = [
    dict(
        active=0,
        pad={"t": 40},
        steps=slider_steps,
    )
]

# Axes as before
fig.update_xaxes(tickangle=-45, automargin=True,
                 tickfont=dict(size=8, family="Arial Black"))
fig.update_yaxes(automargin=True,
                 tickfont=dict(size=8, family="Arial Black"))

# Layout + slider
fig.update_layout(
    title=dict(
        text="<b>Correlation Heatmaps for each service</b>",
        x=0.5,
        xanchor="center",
    ),
    height=850,
    width=950,
    sliders=sliders,
)

fig.show()


Near +1 (red) → strong positive correlation

Near −1 (blue) → strong negative correlation

Near 0 (white/light) → no linear correlation

In [207]:
# #SCATTER PLOT

# import plotly.express as px

# fig = px.scatter(
#     services,
#     x="staff_morale",
#     y="patient_satisfaction",
#     color="service",
#     title="Patient Satisfaction vs Staff Morale",
#     labels={
#         "staff_morale": "Staff Morale",
#         "patient_satisfaction": "Patient Satisfaction"
#     }
# )

# fig.show()


In [208]:
import plotly.graph_objects as go

y_col = "patient_satisfaction"
attr_cols = [
    "available_beds",
    "staff_morale",
    "patients_request",
    "patients_admitted",
    "patients_refused"
]

services_list = services["service"].unique()
initial_attr = "staff_morale"   # default X

# Helper to make nicer axis titles
def pretty(col):
    return col.replace("_", " ").title()

# --- Build figure: one trace per service (for initial attribute) ---
fig = go.Figure()

for service in services_list:
    df_s = services[services["service"] == service]
    fig.add_trace(
        go.Scatter(
            x=df_s[initial_attr],
            y=df_s[y_col],
            mode="markers",
            name=service
        )
    )

# --- Dropdown 1: choose X attribute ---
attr_buttons = []
for attr in attr_cols:
    # new x-data for each trace, in the same order as traces were added
    new_x = []
    for service in services_list:
        df_s = services[services["service"] == service]
        new_x.append(df_s[attr])

    attr_buttons.append(
        dict(
            label=pretty(attr),
            method="update",
            args=[
                {"x": new_x},  # update x for all traces
                {"xaxis": {"title": pretty(attr)}}
            ]
        )
    )

# --- Dropdown 2: choose service(s) ---
service_buttons = []

# Show all
service_buttons.append(
    dict(
        label="All services",
        method="update",
        args=[{"visible": [True] * len(services_list)}]
    )
)

# One service at a time
for i, service in enumerate(services_list):
    vis = [False] * len(services_list)
    vis[i] = True
    service_buttons.append(
        dict(
            label=service,
            method="update",
            args=[{"visible": vis}]
        )
    )

# --- Layout: put dropdowns above the plot, not overlapping ---
fig.update_layout(
    title="Patient Satisfaction vs Selected Attribute",
    xaxis_title=pretty(initial_attr),
    yaxis_title="Patient Satisfaction",
    updatemenus=[
        # Attribute selector
        dict(
            buttons=attr_buttons,
            direction="down",
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.25,
            yanchor="top"
        ),
        # Service selector
        dict(
            buttons=service_buttons,
            direction="down",
            showactive=True,
            x=0.38,
            xanchor="left",
            y=1.25,
            yanchor="top"
        )
    ],
    margin=dict(t=150)
)

fig.show()


In [ ]:
# import numpy as np
# import plotly.graph_objects as go

# y_col = "patient_satisfaction"
# attr_cols = [
#     "available_beds",
#     "staff_morale",
#     "patients_request",
#     "patients_admitted",
#     "patients_refused"
# ]

# services_list = services["service"].unique()
# initial_attr = "staff_morale"   # default X

# def pretty(col):
#     return col.replace("_", " ").title()

# # ---- Helper: regression line ----
# def fit_line(x, y):
#     x = np.asarray(x, dtype=float)
#     y = np.asarray(y, dtype=float)
#     m, b = np.polyfit(x, y, 1)
#     xs = np.linspace(x.min(), x.max(), 50)
#     ys = m * xs + b
#     return xs, ys


# # ---- Build figure: Scatter + regression line for each service ----
# fig = go.Figure()

# for service in services_list:
#     df_s = services[services["service"] == service]

#     # Scatter points
#     fig.add_trace(go.Scatter(
#         x=df_s[initial_attr],
#         y=df_s[y_col],
#         mode="markers",
#         name=service
#     ))

#     # Regression line
#     xs, ys = fit_line(df_s[initial_attr], df_s[y_col])
#     fig.add_trace(go.Scatter(
#         x=xs,
#         y=ys,
#         mode="lines",
#         name=f"{service} (trend)",
#         showlegend=False,
#         line=dict(dash="dash")
#     ))


# # ---- Dropdown 1: X attribute ----
# attr_buttons = []
# for attr in attr_cols:
#     new_x = []
#     new_y = []

#     for service in services_list:
#         df_s = services[services["service"] == service]

#         # markers
#         new_x.append(df_s[attr])
#         new_y.append(None)   # keep existing y

#         # regression line
#         xs, ys = fit_line(df_s[attr], df_s[y_col])
#         new_x.append(xs)
#         new_y.append(ys)

#     attr_buttons.append(dict(
#         label=pretty(attr),
#         method="update",
#         args=[
#             {"x": new_x, "y": new_y},
#             {"xaxis": {"title": pretty(attr)}}
#         ]
#     ))


# # ---- Dropdown 2: Service selector (handles scatter + line) ----
# service_buttons = []

# # All services
# service_buttons.append(dict(
#     label="All services",
#     method="update",
#     args=[{"visible": [True, True] * len(services_list)}]
# ))

# # Single service view
# for i, service in enumerate(services_list):
#     vis = [False] * (2 * len(services_list))
#     vis[2*i] = True       # scatter
#     vis[2*i + 1] = True   # regression line

#     service_buttons.append(dict(
#         label=service,
#         method="update",
#         args=[{"visible": vis}]
#     ))


# # ---- Layout ----
# fig.update_layout(
#     title="Patient Satisfaction vs Selected Attribute",
#     xaxis_title=pretty(initial_attr),
#     yaxis_title="Patient Satisfaction",
#     updatemenus=[
#         dict(
#             buttons=attr_buttons,
#             direction="down",
#             showactive=True,
#             x=0.0, xanchor="left",
#             y=1.25, yanchor="top"
#         ),
#         dict(
#             buttons=service_buttons,
#             direction="down",
#             showactive=True,
#             x=0.38, xanchor="left",
#             y=1.25, yanchor="top"
#         )
#     ],
#     margin=dict(t=150)
# )

# fig.show()
